In [0]:
import sys
import logging
import time
import numpy as np
import pandas as pd
import os.path

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import BaggingClassifier

from xgboost import XGBClassifier

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.regularizers import l1_l2
import pickle

Using TensorFlow backend.


In [0]:
def elapsed_time(start_time, end_time):
    elapsed_sec = end_time - start_time
    h = int(elapsed_sec / (60 * 60))
    m = int((elapsed_sec % (60 * 60)) / 60)
    s = int(elapsed_sec % 60)
    return "{}:{:>02}:{:>02}".format(h, m, s)

In [0]:
def evaluate(y, y_pred):
    logloss = log_loss(y, y_pred)
    return logloss

In [0]:
def load_data(train_data_path='data/train.csv', test_data_path = 'data/test.csv'):
    train_df = pd.read_csv(train_data_path, sep=',', index_col=0, header=0)
    test_df = pd.read_csv(test_data_path, sep=',', index_col=0, header=0)
    
    train_df['target'] = train_df['target'].str[-1].astype(int) - 1
        
    return train_df, test_df

In [0]:
def process_data(train_df, test_df, ylabel='target', standarization=False, discretization=False, transform=None):
    numerical_features = train_df.columns

    if standarization:
        standarized_features = numerical_features
        standarize_feature(train_df, test_df, standarized_features)
        
    if discretization:
        discretized_features = numerical_features
        discretize_feature(train_df, test_df, discretized_features, num_bins=10, how='equal_freq')
    
    X = train_df.drop(ylabel, axis=1).values
    y = train_df[ylabel].values
    X_submission = test_df.values
    
    if transform == 'log':
        X = np.log1p(X)
        X_submission = np.log1p(X_submission)
    elif transform == 'sqrt':
        X = np.sqrt(X + 3.0 / 8)
        X_submission = np.sqrt(X_submission + 3.0 / 8)
    elif transform == 'pca':
        pca = PCA(n_components=3).fit(X)
        X = pca.transform(X)
        X_submission = pca.transform(X_submission)
    elif transform == 'tsne':
        tsne = TSNE(n_components=3).fit(X)
        X = tsne.transform(X)
        X_submission = tsne.transform(X_submission)
    elif transform == 'pca+':
        pca = PCA(n_components=3).fit(X)
        X = np.hstack((X, pca.transform(X)))
        X_submission = np.hstack((X, pca.transform(X)))
    elif transform == 'tsne+':
        tsne = TSNE(n_components=3).fit(X)
        X = np.hstack((X, tsne.transform(X)))
        X_submission = np.hstack((X_submission, tsne.transform(X_submission)))        
    return X, y, X_submission

In [0]:
def model_CV_train(model, X, y, X_submission, n_classes, n_folds=5):
    summary = {}

    skf = list(StratifiedKFold(n_folds, random_state=0).split(X, y))
    
    stack_train = np.zeros((X.shape[0], n_classes))
    stack_test = np.zeros((X_submission.shape[0], n_classes))
    
#     print("Model :" model)

    avg_logloss = 0

    stack_test_model = np.zeros((X_submission.shape[0], n_classes, len(skf)))
    for j, (train_idx, test_idx) in enumerate(skf):
        print ("  Fold %d" % j)
        X_train = X[train_idx]
        y_train = y[train_idx]
        X_test = X[test_idx]
        y_test = y[test_idx]

        model.fit(X_train, y_train)

        y_test_pred = model.predict_proba(X_test)          
        stack_train[test_idx, :] = y_test_pred

        logloss = evaluate(y_test, y_test_pred)
        avg_logloss += logloss
        print ("  logloss: %f" % logloss)

        y_submission_pred = model.predict_proba(X_submission)           
        stack_test_model[:, :, j] = y_submission_pred

    avg_logloss = avg_logloss / n_folds
    print ("model average logloss: %f" % avg_logloss)
    summary = avg_logloss

    stack_test[:, :] = stack_test_model.mean(axis=2)

    return stack_train, stack_test, summary

In [0]:
start_time = time.time()

logging.basicConfig(level=logging.DEBUG,
                    format='[%(asctime)s]: %(message)s ',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    stream=sys.stdout,
                    filemode="w"
                    )
# load data
logging.info('Load data')
train_df, test_df = load_data(train_data_path='applied/otto/train.csv', test_data_path='applied/otto/test.csv')
X, y, X_submission = process_data(train_df, test_df, transform='log')

[2020-02-27 02:23:45]: Load data 
[2020-02-27 02:23:49]: NumExpr defaulting to 8 threads. 


In [0]:
# Tune parameters

model =  ExtraTreesClassifier(n_jobs=-1,random_state = 42)
parameters = {
    'n_estimators': [100, 200], 
     'min_samples_split': [25], 
     'min_samples_leaf': [35], 
    #  'max_features': [150]
    }
classifier=GridSearchCV(estimator=model, param_grid=parameters, scoring='neg_log_loss', verbose=1, n_jobs=-1, cv=5);  

In [0]:
classifier.fit(X, y);

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:  1.8min finished


In [0]:
print(classifier.cv_results_.keys())
print('best_params: ',classifier.best_params_)
print('best_score: ',classifier.best_score_)
for i in range(len(classifier.cv_results_['params'])):
  print('{}, {}'.format(classifier.cv_results_['params'][i], classifier.cv_results_['mean_test_score'][i]))

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_min_samples_leaf', 'param_min_samples_split', 'param_n_estimators', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])
best_params:  {'min_samples_leaf': 35, 'min_samples_split': 25, 'n_estimators': 200}
best_score:  -0.8800639611665805
{'min_samples_leaf': 35, 'min_samples_split': 25, 'n_estimators': 100}, -0.8812342983806699
{'min_samples_leaf': 35, 'min_samples_split': 25, 'n_estimators': 200}, -0.8800639611665805


In [0]:
best_model = ExtraTreesClassifier(n_estimators = classifier.best_params_['n_estimators'],
                                min_samples_split = classifier.best_params_['min_samples_split'],
                                min_samples_leaf = classifier.best_params_['min_samples_leaf'],
                                random_state=42,
                                n_jobs = -1 ) 

In [0]:
model_3_path = "applied/otto/model_3.csv"
n_estimators = []
min_samples_split = []
min_samples_leaf = []
mean_test_score = []
if os.path.exists(model_3_path):
    previous_gridsearch = pd.read_csv(model_3_path, sep=',', header=0)
    for i in range(previous_gridsearch.values.shape[0]):
        n_estimators.append(previous_gridsearch.values[i][0])
        min_samples_split.append(previous_gridsearch.values[i][1])
        min_samples_leaf.append(previous_gridsearch.values[i][2])
        mean_test_score.append(previous_gridsearch.values[i][3])

In [0]:
submit = pd.DataFrame()

for i in range(len(classifier.cv_results_['params'])):
    n_estimators.append(classifier.cv_results_['params'][i]['n_estimators'])
    min_samples_split.append(classifier.cv_results_['params'][i]['min_samples_split'])
    min_samples_leaf.append(classifier.cv_results_['params'][i]['min_samples_leaf'])
    mean_test_score.append(classifier.cv_results_['mean_test_score'][i])


  # print(classifier.cv_results_['params'][i]['n_estimators'])
submit["n_estimators"] = n_estimators
submit["min_samples_split"] = min_samples_split
submit["min_samples_leaf"] = min_samples_leaf
submit["mean_test_score"] = mean_test_score
submit.to_csv("applied/otto/model_3.csv", index = False)

In [0]:
# Train model using best parameters
train_models_pred, test_models_pred, summary = model_CV_train(best_model, X, y, X_submission, n_classes=9, n_folds=5)
end_time = time.time()
logging.info("Run complete: %s elapsed" % elapsed_time(start_time, end_time))

  Fold 0
  logloss: 0.887348
  Fold 1
  logloss: 0.887161
  Fold 2
  logloss: 0.879103
  Fold 3
  logloss: 0.870750
  Fold 4
  logloss: 0.875955
model average logloss: 0.880063
[2020-02-27 02:38:01]: Run complete: 0:14:16 elapsed 


In [0]:
pkl_filename = "applied/otto/train_models_pred_model_3.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(train_models_pred, file)
pkl_filename = "applied/otto/test_models_pred_model_3.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(test_models_pred, file)
# with open(pkl_filename, 'rb') as file:
#   test = pickle.load(file)
# print(test.shape)
pkl_filename = "applied/otto/summary_model_3.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(summary, file)